In [1]:
import pandas as pd

import funcoes

In [2]:
df = pd.read_csv('C:/Users/tales/OneDrive/Documentos/GitHub/analise_dados_crimes_seatle/data/raw/SPD_Crime_Data__2008-Present_20250223.csv', sep=',')
dff = df.copy() #preservando o bruto

In [12]:
dff = funcoes.colunas_lower(df)

In [4]:
dff.columns

Index(['report number', 'offense id', 'offense start datetime',
       'offense end datetime', 'report datetime', 'group a b',
       'crime against category', 'offense parent group', 'offense',
       'offense code', 'precinct', 'sector', 'beat', 'mcpp',
       '100 block address', 'longitude', 'latitude'],
      dtype='object')

In [ ]:
df = df.drop(425006)

In [ ]:
dff['offense start datetime'] = funcoes.transformando_colunas_em_datetime(dff, 'offense start datetime')
dff['offense end datetime'] = pd.to_datetime(dff['offense end datetime'])
dff['report datetime'] = funcoes.transformando_colunas_em_datetime(dff, 'report datetime')

In [7]:
dff['precinct'].value_counts()

precinct
N            377929
W            312720
E            187634
S            170513
SW           118191
UNKNOWN        7470
w                 5
OOJ               1
NORTHGATE         1
Name: count, dtype: int64